In [2]:
pip install pyro-ppl

  Obtaining dependency information for pyro-ppl from https://files.pythonhosted.org/packages/f2/93/59bced321ede6eeb60061f156df8aae3f4832127fe97f4e86c567ad3b9cc/pyro_ppl-1.8.6-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.8/732.8 kB 27.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install yfinance

  Obtaining dependency information for yfinance from https://files.pythonhosted.org/packages/58/f7/a966b800b49cb5379e348bf0717df481051714a99ece15289718562f6952/yfinance-0.2.35-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 54.1 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 kB 7.1 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.0-cp310-cp310-linux_x86_64.whl size=272555 sha256=4d4919142444f0d342dccab5954926c6aaef4653b026a00e1fbe8caa65039039
  Stored in directory: /root/.cache/pip/wheels/c7/70/ad/212867e96e7004265a69c4aa5dcff00a95f547a67ba26e7e76
Successfully built peewee
Note: you may need to restart the kernel to use updated packages.


In [5]:
 pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [6]:
#imports 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from IPython import display
import os
from PIL import Image
from torch.utils.data.dataset import Dataset
import scipy
import torch.nn.functional as nnf
from torch.optim import SGD 

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [8]:
import pyro
from pyro.distributions import Normal, Categorical
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
import yfinance as yf
import matplotlib.pyplot as plt
import time
import fastprogress
import seaborn as sns
from pyro.nn import PyroModule, PyroSample

In [ ]:

def train(dataloader, optimizer, model, loss_fn, device, master_bar):
    """Run one training epoch.

    Args:
        dataloader (DataLoader): Torch DataLoader object to load data
        optimizer: Torch optimizer object
        model (nn.Module): Torch model to train
        loss_fn: Torch loss function
        device (torch.device): Torch device to use for training
        master_bar (fastprogress.master_bar): Will be iterated over for each
            epoch to draw batches and display training progress

    Returns:
        float, float: Mean loss of this epoch, fraction of correct predictions
            on training set (accuracy)
    """
    epoch_loss = []

    for x, y in fastprogress.progress_bar(dataloader, parent=master_bar):
        optimizer.zero_grad()
        model.train()

        # Forward pass
        y_pred = model(x.to(device).float())
        
        # Compute loss
        loss = loss_fn(y_pred, y.to(device).float())

        # Backward pass
        loss.backward()
        optimizer.step()

        # For plotting the train loss, save it for each sample
        epoch_loss.append(loss.item())

    # Return the mean loss and the accuracy of this epoch
    return np.mean(epoch_loss)


def validate(dataloader, model, loss_fn, device, master_bar):
    """Compute loss, accuracy and confusion matrix on validation set.

    Args:
        dataloader (DataLoader): Torch DataLoader object to load data
        model (nn.Module): Torch model to train
        loss_fn: Torch loss function
        device (torch.device): Torch device to use for training
        master_bar (fastprogress.master_bar): Will be iterated over to draw 
            batches and show validation progress

    Returns:
        float, float, torch.Tensor shape (10,10): Mean loss on validation set, 
            fraction of correct predictions on validation set (accuracy)
    """
    epoch_loss = []  

    model.eval()
    with torch.no_grad():
        for x, y in fastprogress.progress_bar(dataloader, parent=master_bar):
            # make a prediction on validation set
            y_pred = model(x.to(device).float())

            # Compute loss
            loss = loss_fn(y_pred, y.to(device).float())

            # For plotting the train loss, save it for each sample
            epoch_loss.append(loss.item())

    # Return the mean loss, the accuracy and the confusion matrix
    return np.mean(epoch_loss)


def run_training(model, optimizer, loss_function, device, num_epochs, 
                train_dataloader, val_dataloader, verbose=False):
    """Run model training.

    Args:
        model (nn.Module): Torch model to train
        optimizer: Torch optimizer object
        loss_fn: Torch loss function for training
        device (torch.device): Torch device to use for training
        num_epochs (int): Max. number of epochs to train
        train_dataloader (DataLoader): Torch DataLoader object to load the
            training data
        val_dataloader (DataLoader): Torch DataLoader object to load the
            validation data
        early_stopper (EarlyStopper, optional): If passed, model will be trained
            with early stopping. Defaults to None.
        verbose (bool, optional): Print information about model training. 
            Defaults to False.

    Returns:
        list, list, list, list, torch.Tensor shape (10,10): Return list of train
            losses, validation losses, train accuracies, validation accuracies
            per epoch and the confusion matrix evaluated in the last epoch.
    """
    start_time = time.time()
    master_bar = fastprogress.master_bar(range(num_epochs))
    train_losses, val_losses, train_accs, val_accs = [],[],[],[]

    for epoch in master_bar:
        # Train the model
        epoch_train_loss = train(train_dataloader, optimizer, model, 
                                                  loss_function, device, master_bar)
        # Validate the model
        epoch_val_loss = validate(val_dataloader, model, loss_function, device, master_bar)

        # Save loss and acc for plotting
        train_losses.append(epoch_train_loss)
        val_losses.append(epoch_val_loss)
        
        if verbose:
            master_bar.write(f'Train loss: {epoch_train_loss:.2f}, val loss: {epoch_val_loss:.2f}, train acc: {epoch_train_acc:.3f}, val acc {epoch_val_acc:.3f}')

    time_elapsed = np.round(time.time() - start_time, 0).astype(int)
    print(f'Finished training after {time_elapsed} seconds.')
    return train_losses, val_losses


def plot(title, label, train_results, val_results, yscale='linear', save_path=None, 
         extra_pt=None, extra_pt_label=None):
    """Plot learning curves.

    Args:
        title (str): Title of plot
        label (str): x-axis label
        train_results (list): Results vector of training of length of number
            of epochs trained. Could be loss or accuracy.
        val_results (list): Results vector of validation of length of number
            of epochs. Could be loss or accuracy.
        yscale (str, optional): Matplotlib.pyplot.yscale parameter. 
            Defaults to 'linear'.
        save_path (str, optional): If passed, figure will be saved at this path.
            Defaults to None.
        extra_pt (tuple, optional): Tuple of length 2, defining x and y coordinate
            of where an additional black dot will be plotted. Defaults to None.
        extra_pt_label (str, optional): Legend label of extra point. Defaults to None.
    """
    
    epoch_array = np.arange(len(train_results)) + 1
    train_label, val_label = "Training "+label.lower(), "Validation "+label.lower()
    
    sns.set(style='ticks')

    plt.plot(epoch_array, train_results, epoch_array, val_results, linestyle='dashed', marker='o')
    legend = ['Train results', 'Validation results']
    
    if extra_pt:
        plt.plot(*extra_pt, 'ok')
    if extra_pt_label:
        legend.append(extra_pt_label)
    # END OF YOUR CODE #
        
    plt.legend(legend)
    plt.xlabel('Epoch')
    plt.ylabel(label)
    plt.yscale(yscale)
    plt.title(title)
    
    sns.despine(trim=True, offset=5)
    plt.title(title, fontsize=15)
    if save_path:
        plt.savefig(str(save_path), bbox_inches='tight')
    plt.show()

In [ ]:
#data import and processing 
crude = yf.Ticker("CL=F")
hist = crude.history(period="24mo")
plt.plot(hist.loc[:,"Close"])






In [ ]:
#normalize data and plot again 
data = hist.loc[:,"Close"]
data_norm = (data - min(data)) / (max(data)-min(data))
plt.plot(data_norm)

In [ ]:
data_norm = list(data_norm)


train_set = data_norm[:453]
valid_set = data_norm[454: ]
print('Proportion of train_set : {:.2f}%'.format(len(train_set)/len(data_norm)))
print('Proportion of valid_set : {:.2f}%'.format(len(valid_set)/len(data_norm)))



In [ ]:
from numpy import array
 
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# split into samples
train_x,train_y  = split_sequence(train_set, 3)
valid_x,valid_y  = split_sequence(valid_set, 3) 
# summarize the data
for i in range(5):
    print(train_x[i], train_y[i])




In [ ]:
class myDataset(Dataset):
    def __init__(self,feature,target):
        self.feature = feature
        self.target = target
    
    def __len__(self):
        return len(self.feature)
    
    def __getitem__(self,idx):
        item = self.feature[idx]
        label = self.target[idx]
        
        return item,label



In [ ]:
class ts_CNN(nn.Module):
    
    def __init__(self):
        super(ts_CNN, self).__init__()
        
        self.feature= nn.Sequential(
        nn.Conv1d(in_channels = 3, out_channels=64, kernel_size = 1),
        nn.ReLU(),
        )
        
        self.result = nn.Sequential(
        nn.Linear(1*64,50),
        nn.ReLU(),
        nn.Linear(50,1))
        
    def forward(self,x):
        
        x = self.feature(x)
        x = torch.flatten(x,1)
        x = self.result(x)
        
        return x.float()

In [ ]:
class BNN(PyroModule):
    def __init__(self, in_dim=1, out_dim=1, hid_dim=10, n_hid_layers=5, prior_scale=5.):
        super().__init__()
        assert in_dim > 0 and out_dim > 0 and hid_dim > 0 and n_hid_layers > 0 
        #self.layer_sizes = [in_dim] + n_hid_layers * [hid_dim] + [out_dim]
        self.layer_1 = PyroModule[nn.Conv1d](in_channels = 3, out_channels=64, kernel_size = 1)
        self.activation = PyroModule[nn.ReLU]()
        self.linear_1 = PyroModule[nn.Linear](1*64,50)
        self.linear_2 = PyroModule[nn.Linear](50,1)
        
        self.layer_1.weight = PyroSample(dist.Normal(0, 1)).expand(3,64).to_event(2)
        self.layer_1.bias = PyroSample(dist.Normal(0, 1)).expand(64).to_event(1)
        self.linear_1.weight = PyroSample(dist.Normal(0, 1)).expand(64,50).to_event(2)
        self.linear_1.bias = PyroSample(dist.Normal(0, 1)).expand(50).to_event(1)
        self.linear_2.weight = PyroSampe(dist.Normal(0, 1)).expand(50,1).to_event(2)
        self.linear_2.bias = PyroSample(dist.Normal(0, 1)).expand(1).to_event(1)
    
    def forward(self, x):
        
    

In [ ]:
from pyro.nn.module import to_pyro_module_
import pyro.distributions as dist

model = nn.Sequential(
    nn.Linear(28 * 28, 100),
    nn.Sigmoid(),
    nn.Linear(100, 100),
    nn.Sigmoid(),
    nn.Linear(100, 10),
)
to_pyro_module_(model)
assert isinstance(model, PyroModule[nn.Sequential])
assert isinstance(model[0], PyroModule[nn.Linear])

# Now we can attempt to be fully Bayesian:
for m in model.modules():
    for name, value in list(m.named_parameters(recurse=False)):
        setattr(m, name, PyroSample(prior=dist.Normal(0, 1)
                                              .expand(value.shape)
                                              .to_event(value.dim())))
        print(value.shape)
guide = AutoDiagonalNormal(model)



In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = ts_CNN()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss = nn.MSELoss()

In [ ]:
train_data = myDataset(train_x.reshape(train_x.shape[0],train_x.shape[1],1),train_y)
valid_data = myDataset(valid_x.reshape(valid_x.shape[0],valid_x.shape[1],1),valid_y)
train_loader = torch.utils.data.DataLoader(train_data,batch_size=2,shuffle=False)
valid_loader = torch.utils.data.DataLoader(valid_data,batch_size=2,shuffle=False)

In [ ]:
num_epochs = 100
train_losses, val_losses = run_training(model, optimizer, 
                loss, device, num_epochs, train_loader, valid_loader,verbose=False)

In [ ]:
plot('CNN_results', 'Loss', train_losses, val_losses, yscale='linear', save_path=None, 
         extra_pt=None, extra_pt_label=None)

In [ ]:
target_x , target_y = split_sequence(valid_set,3)
inputs = target_x.reshape(target_x.shape[0],target_x.shape[1],1)



In [ ]:
model.eval()
prediction = []
batch_size = 2
iterations =  int(inputs.shape[0]/2)

for i in range(iterations):
    preds = model(torch.tensor(inputs[batch_size*i:batch_size*(i+1)]).float().to(device))
    prediction.append(preds.detach().cpu().tolist())

In [ ]:
prediction = np.array(prediction).ravel()

In [ ]:
len(inputs)

In [ ]:

fig, ax = plt.subplots(1, 2,figsize=(11,4))
ax[0].set_title('predicted one')
ax[0].plot(prediction)
ax[1].set_title('real one')
ax[1].plot(target_y)
plt.show()

